In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [75]:
def read_conllu(path):
    sentences = []
    sentence = []

    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()

            # Sentence boundary
            if not line:
                if sentence:
                    sentences.append(sentence)
                    sentence = []
                continue

            # Skip comments
            if line.startswith("#"):
                continue

            cols = line.split("\t")

            # Skip multi-word tokens like "1-2 don't"
            if "-" in cols[0]:
                continue

            word = cols[1].lower()
            upos = cols[3]

            sentence.append((word, upos))

    if sentence:
        sentences.append(sentence)

    return sentences


In [80]:
from collections import Counter

def train_hmm(train_sentences, threshold=1):
    counter = 0
    print("train_sentences")
    while counter < 2:
        print(train_sentences[counter])
        counter += 1
    print()
    # counter = 0
    # while counter < 2:
    #     print(test_sentences[counter])
    #     counter += 1

    word_counts = Counter(
    word for sent in train_sentences for word, _ in sent
    )
    print("Most Common Words")
    print(word_counts.most_common(10))

    
    UNK = "<UNK>"
    threshold = 1 
    train_sentences_unk = []

    for sent in train_sentences:

        new_sent = []

        for word, tag in sent:

            if word_counts[word] <= threshold:
                new_word = UNK
            else:
                new_word = word

            new_sent.append((new_word, tag))

        train_sentences_unk.append(new_sent)

        
    vocab = set()

    for sent in train_sentences_unk:
        for word, tag in sent:
            vocab.add(word)

    print("Vocabulary size:", len(vocab))
    print("<UNK> in vocabulary:", UNK in vocab)


    #Emission probabilities
    emission_counts = Counter()
    tag_counts_emission = Counter()

    for sent in train_sentences_unk:
        for word, tag in sent:
            emission_counts[(tag, word)] += 1
            tag_counts_emission[tag] += 1
    # print(len(tag_counts_emission))
    emission_probs = {}

    for (tag, word), count in emission_counts.items():
        emission_probs[(tag, word)] = count / tag_counts_emission[tag]
    # print(emission_probs)
    tags = list(tag_counts_emission.keys())

    
    #Initial Probabilities (P(tag/START))


    initial_tag_counts = Counter()
    total_sentences = len(train_sentences_unk)

    for sent in train_sentences_unk:
        #check the tag of first word of each sentence
        first_tag = sent[0][1]
        initial_tag_counts[first_tag] += 1
    # print(initial_tag_counts)
    # print()
    #calculate prob that a tag occurs at the start
    # initial_probs = {
    #     tag: count / total_sentences
    #     for tag, count in initial_tag_counts.items()
    # }
    initial_probs = {
    tag: initial_tag_counts.get(tag, 0) / total_sentences
    for tag in tags
    }



    sorted_data = dict(sorted(initial_probs.items(), key=lambda item: item[1]))
    print(f"initial_probs(sorted_data): {sorted_data}")
    # print(initial_probs)  



    #Transition probabilities
    transition_counts = Counter()
    tag_counts = Counter()   # denominator

    for sent in train_sentences_unk:
        for i in range(len(sent) - 1):
            tag_i = sent[i][1]
            tag_j = sent[i + 1][1]

            transition_counts[(tag_i, tag_j)] += 1
            tag_counts[tag_i] += 1
    transition_probs = {}

    for (tag_i, tag_j), count in transition_counts.items():
        transition_probs[(tag_i, tag_j)] = count / tag_counts[tag_i]

    # print(transition_probs)


    return initial_probs, transition_probs, emission_probs, vocab, tags

    
        
        

In [81]:
def basic_checks(initial_probs,transition_probs,emission_probs):
#basic checks
    print(sum(initial_probs.values()))
    print(sum(
        prob for (t1, _), prob in transition_probs.items()
        if t1 == "NOUN"
    )
    )
    print(sum(
        prob for (t, _), prob in emission_probs.items()
        if t == "NOUN"
    )
    )

    # total
    # for (tag, word), prob in emission_probs.items():
    #     if tag == "PUNCT":
    #         # print(f"word={word} prob={prob}")
    #         total=total+prob
    # print(total)